In [1]:
import pandas as pd
import numpy as np
import datetime, time

import matplotlib.pyplot as plt

In [2]:
def allsatsundays(start_date, end_date):
    day_sat = pd.date_range(start=start_date, end=end_date, 
                         freq='W-SAT').strftime('%Y-%m-%d').tolist()
    day_sun = pd.date_range(start=start_date, end=end_date, 
                         freq='W-SUN').strftime('%Y-%m-%d').tolist()
    days = day_sat+day_sun
    days.sort()
    return days

def allsatsundays(start_date, end_date):
    day_sat = pd.date_range(start=start_date, end=end_date, 
                         freq='W-SAT').strftime('%Y-%m-%d').tolist()
    day_sun = pd.date_range(start=start_date, end=end_date, 
                         freq='W-SUN').strftime('%Y-%m-%d').tolist()
    days = day_sat+day_sun
    days.sort()
    return days

def get_stock_market_holydays(filename, start_date, end_date):
    df = pd.read_csv(filename)
    kor_days = df['KOR'][df['KOR'].notnull()].values.tolist()
    usa_days = df['USA'][df['USA'].notnull()].values.tolist() # Nan value를 제외한 값을 리스트로 변환

    holydays = allsatsundays(start_date, end_date)
    
    kor_days = sorted(set(kor_days + holydays))
    usa_days = sorted(set(usa_days + holydays))
    
    return kor_days, usa_days

def get_seq_data(filename, holydays):
    df = pd.read_csv(filename)
    df_seq = df[~df['date'].isin(holydays)].copy()
    df_seq.dropna(subset=df.columns[1:], how='all', inplace=True)
    return df_seq

In [3]:
def classify_data(df, col_name): # column 데이터를 9개로 분류
    col_index = df.columns.tolist()
    for i, col in enumerate(col_name):
        mu = df[col].mean()
        sigma = df[col].std()
        df_min = df[col].min()
        df_max = df[col].max()
        cut_div = [df_min, mu - 3*sigma, mu - 2*sigma, mu - sigma, mu - 0.5*sigma, 
                   mu + 0.5*sigma, mu + sigma, mu + 2*sigma, mu + 3*sigma, df_max ]
        ix = i + col_index.index(col)
        df_new = pd.cut(df[col], cut_div, labels=[1, 2, 3, 4, 5, 6, 7, 8, 9])
        df.insert(ix+1, col+'_class', df_new)
        df[[col, col+'_class']].head()
        
    return df

In [4]:
def consecutive_count(df_sr, idx): # series, idx: row index 여기서부터 역산으로 추출
    # 매도(-), 매수(+) 연속일 계산
    if idx >= 29: # 최대 20개까지만 확인
        df_sel = df_sr[idx-29:idx+1][::-1]
    else:
        df_sel = df_sr[:idx+1][::-1]
    sr = df_sel.apply(lambda x : 1 if (x > 0) else -1)
    p_sum = 0
    m_sum = 0
    z_sum = 0
    for i, num in enumerate(sr):
        if (num == 0) & (i == z_sum) :
            z_sum += 1
            continue
        elif (num == 1) & (i == p_sum) :
            p_sum += 1
            continue
        elif (num == -1) & (i == m_sum) :
            m_sum += 1
            continue   
        else :
            break

    if p_sum > 0 : 
        return p_sum
    elif m_sum > 0 :
        return -m_sum
    else :
        return z_sum

In [5]:
def get_dayofweek(df):
    # insert 요일 column: 1: monday, 5: friday
    df['date'] = df['date'].apply(lambda x : datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dayofweek = df['date'].dt.dayofweek + 1 # 1: monday, 5: Friday
    df.insert(1, 'dayofweek', df_dayofweek)
    
    return df

In [11]:
def consequtive_days(df, col_names):
    for col in col_names:
        df_temp = df[col].apply(lambda x : 1 if (x > 0) else -1)
        count = []
        for idx in range(len(df_temp)):
            count.append(consecutive_count(df_temp, idx))

        df[col+'_days'] = count  # 매수, 매도 거래 연속일
        
    return df

In [15]:
def get_weight_ratio(df, investors, investor_sum): # 투자자별 매수 매도 비중 비율
    df['total'] = df[investors_sum].abs().sum(axis=1).to_frame()
    for col in investors:
        df[col+'_ratio'] = df[col]/df['total']
        
    return df    

In [6]:
filename = 'https://raw.githubusercontent.com/fasthill/'\
            'factors_affecting_stock_price/main/data/stock_market_holydays/stock_market_holidays.csv'
start_date = '2020-01-01'
end_date = '2022-12-31'

kor_days, usa_days = get_stock_market_holydays(filename, start_date, end_date) # get stock market holydays

In [7]:
filename = '../data/data_hyunmotor.csv'
df = get_seq_data(filename, kor_days)

In [8]:
# get day of week
df = get_dayofweek(df)

In [9]:
df =  df[:-1]
df.tail()

,date,dayofweek,retail,retail_cr,retail_days,foreigner,foreigner_cr,foreigner_days,institution,institution_cr,...,rc4_pcr,open,high,low,close,close_cr,vol,vol_cr,high_time,low_time
919,2022-07-08,5,-20479.0,NaN,NaN,15617.0,NaN,NaN,3853.0,NaN,...,NaN,177500.0,180500.0,177000.0,178000.0,1.71,147130.0,NaN,NaN,NaN
922,2022-07-11,1,-10749.0,NaN,NaN,11123.0,NaN,NaN,-719.0,NaN,...,NaN,180000.0,182000.0,179000.0,179000.0,0.56,119200.0,NaN,NaN,NaN
923,2022-07-12,2,868.0,NaN,NaN,3282.0,NaN,NaN,-3835.0,NaN,...,NaN,180000.0,181500.0,177000.0,179000.0,0.00,96440.0,NaN,NaN,NaN
924,2022-07-13,3,-40414.0,NaN,NaN,18096.0,NaN,NaN,22755.0,NaN,...,NaN,180000.0,185000.0,178500.0,183500.0,2.51,172566.0,NaN,NaN,NaN
925,2022-07-14,4,13810.0,NaN,NaN,9619.0,NaN,NaN,-23410.0,NaN,...,NaN,183000.0,184500.0,180500.0,182500.0,-0.54,142369.0,NaN,NaN,NaN


In [10]:
investors = ['retail', 'foreigner', 'institution', 'financial', 'invtrust', 'pension',
             'privequity', 'bank',  'insurance', 'financeetc', 'corporateetc', 'foreigneretc']
df = classify_data(df, investors) # column data catagorizing

,date,dayofweek,retail,retail_class,retail_cr,retail_days,foreigner,foreigner_class,foreigner_cr,foreigner_days,...,rc4_pcr,open,high,low,close,close_cr,vol,vol_cr,high_time,low_time
1,2020-01-02,4,21980.0,5,NaN,NaN,-14050.0,5,NaN,NaN,...,NaN,121000.0,121000.0,118000.0,118000.0,-2.07,66205.0,NaN,NaN,NaN
2,2020-01-03,5,42651.0,6,NaN,NaN,-27939.0,4,NaN,NaN,...,NaN,118500.0,119000.0,115000.0,116000.0,-1.69,133657.0,NaN,NaN,NaN
5,2020-01-06,1,2658.0,5,NaN,NaN,-1144.0,5,NaN,NaN,...,NaN,116000.0,117000.0,115500.0,116000.0,0.00,38907.0,NaN,NaN,NaN
6,2020-01-07,2,13532.0,5,NaN,NaN,-5817.0,5,NaN,NaN,...,NaN,116500.0,117000.0,115500.0,115500.0,-0.43,78229.0,NaN,NaN,NaN
7,2020-01-08,3,17848.0,5,NaN,NaN,-1415.0,5,NaN,NaN,...,NaN,115000.0,115500.0,111500.0,112000.0,-3.03,117813.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
919,2022-07-08,5,-20479.0,4,NaN,NaN,15617.0,6,NaN,NaN,...,NaN,177500.0,180500.0,177000.0,178000.0,1.71,147130.0,NaN,NaN,NaN
922,2022-07-11,1,-10749.0,5,NaN,NaN,11123.0,6,NaN,NaN,...,NaN,180000.0,182000.0,179000.0,179000.0,0.56,119200.0,NaN,NaN,NaN
923,2022-07-12,2,868.0,5,NaN,NaN,3282.0,5,NaN,NaN,...,NaN,180000.0,181500.0,177000.0,179000.0,0.00,96440.0,NaN,NaN,NaN
924,2022-07-13,3,-40414.0,4,NaN,NaN,18096.0,6,NaN,NaN,...,NaN,180000.0,185000.0,178500.0,183500.0,2.51,172566.0,NaN,NaN,NaN


In [12]:
# get consecutive days
df = consequtive_days(df, investors)

In [14]:
df.head()

,date,dayofweek,retail,retail_class,retail_cr,retail_days,foreigner,foreigner_class,foreigner_cr,foreigner_days,...,rc4_pcr,open,high,low,close,close_cr,vol,vol_cr,high_time,low_time
1,2020-01-02,4,21980.0,5,NaN,1,-14050.0,5,NaN,-1,...,NaN,121000.0,121000.0,118000.0,118000.0,-2.07,66205.0,NaN,NaN,NaN
2,2020-01-03,5,42651.0,6,NaN,2,-27939.0,4,NaN,-2,...,NaN,118500.0,119000.0,115000.0,116000.0,-1.69,133657.0,NaN,NaN,NaN
5,2020-01-06,1,2658.0,5,NaN,3,-1144.0,5,NaN,-3,...,NaN,116000.0,117000.0,115500.0,116000.0,0.00,38907.0,NaN,NaN,NaN
6,2020-01-07,2,13532.0,5,NaN,4,-5817.0,5,NaN,-4,...,NaN,116500.0,117000.0,115500.0,115500.0,-0.43,78229.0,NaN,NaN,NaN
7,2020-01-08,3,17848.0,5,NaN,5,-1415.0,5,NaN,-5,...,NaN,115000.0,115500.0,111500.0,112000.0,-3.03,117813.0,NaN,NaN,NaN


In [17]:
# get investors amount relative ratio
investors_sum = ['retail', 'foreigner', 'financial', 'invtrust', 'pension',
         'privequity', 'bank',  'insurance', 'financeetc', 'corporateetc', 'foreigneretc']
df = get_weight_ratio(df, investors, investors_sum)

In [18]:
df.head()

,date,dayofweek,retail,retail_class,retail_cr,retail_days,foreigner,foreigner_class,foreigner_cr,foreigner_days,...,institution_ratio,financial_ratio,invtrust_ratio,pension_ratio,privequity_ratio,bank_ratio,insurance_ratio,financeetc_ratio,corporateetc_ratio,foreigneretc_ratio
1,2020-01-02,4,21980.0,5,NaN,1,-14050.0,5,NaN,-1,...,-0.231603,-0.047868,-0.009784,-0.183071,-0.000405,0.000921,0.006541,0.002064,0.084755,0.000737
2,2020-01-03,5,42651.0,6,NaN,2,-27939.0,4,NaN,-2,...,-0.215789,-0.137546,-0.006988,-0.037219,-0.017648,-0.000437,-0.015797,-0.000173,0.064623,0.001526
5,2020-01-06,1,2658.0,5,NaN,3,-1144.0,5,NaN,-3,...,-0.231513,-0.421401,-0.025668,0.061366,0.005790,0.000000,0.148901,-0.000410,0.162807,-0.000319
6,2020-01-07,2,13532.0,5,NaN,4,-5817.0,5,NaN,-4,...,-0.328542,-0.077922,-0.050066,-0.181224,0.002333,0.000086,-0.021749,0.000000,0.107766,-0.001469
7,2020-01-08,3,17848.0,5,NaN,5,-1415.0,5,NaN,-5,...,-0.421158,-0.122701,-0.020340,-0.327383,0.050428,0.000040,-0.000060,-0.001143,0.091509,0.000000


In [ ]:
df_temp = df['retail'].apply(lambda x : 1 if (x > 0) else -1)
count = []
for idx in range(len(df_temp)):
    count.append(consecutive_count(df_temp, idx))

df['retail_days'] = count  # 매수, 매도 거래 연속일

In [ ]:
df.head()

In [ ]:
# 각 주체별 거래 비중 _weight
investors_sum = ['retail', 'foreigner', 'financial', 'invtrust', 'pension',
             'privequity', 'bank',  'insurance', 'financeetc', 'corporateetc', 'foreigneretc']
df['total'] = df[investors_sum].abs().sum(axis=1).to_frame()

In [ ]:
total.head()

In [ ]:
df['retail_ratio'] = df['retail']/df['total']

In [ ]:
df['retail_ratio'].head(20)

In [ ]:
df[investors_sum].abs()

In [ ]:
type(total)

In [ ]:
get_5days_data(df['retail'])

In [ ]:
x = np.arange(len(df['retail']))
plt.scatter(x, df['retail'])

In [ ]:
df.loc[:, ['retail','retail_class', 'foreigner', 'foreigner_class', 'institution', 'institution_class']]

In [ ]:
df.head(15)

In [ ]:
df.tail()

In [ ]:
df_temp = df['insurance'].apply(lambda x : 1 if (x > 0) else -1)
df_temp = df['foreigner'].apply(lambda x : 1 if (x > 0) else -1)

In [ ]:
type(df_temp)

In [ ]:
df_temp[:20][::-1]

In [ ]:
df_temp = [-1, -1, 0, 1, 1, 1, -1]

In [ ]:
x = consecutive_count(df['insurance'][-20:])

In [ ]:
x

In [ ]:
df_count['ori'].apply(pd.Index(df_count['ori']).get_loc)

In [ ]:
df_shift = df['insurance'].shift(-20)

In [ ]:
df_count = pd.DataFrame()

In [ ]:
df_count = pd.concat([df_count, df['insurance'], df['insurance'].shift(10)], axis=1)
df_count.columns = ['ori','shift']

In [ ]:
df_count.head()

In [ ]:
import datetime, time
def get_20days_data(series):
    x_num = np.arange(1, 21)
    for i in range(len(series)-600):
        y = series[i:i+20]
        plt.plot(x_num, y)
        plt.show()
        time.sleep(0.3)

In [ ]:
import datetime, time
def get_5days_data(series):
    x_num = np.arange(1, 6) # 과거 5일 데이터
    for i in range(len(series)-610):
        y = series[i:i+5]
        y = y - y.iloc[0]
        plt.plot(x_num, y)

In [ ]:
df_count['ori'].apply(consecutive_count, args=(pd.Index(df_count['ori']).get_loc))

In [ ]:
df_count['ori'].apply(pd.Index(df_count['ori']).get_loc)

In [ ]:
pd.take()

In [ ]:
y['a'], y['a'].shift()

In [ ]:
y != y.shift()

In [ ]:
c = lambda y: y * (y.groupby((y != y.shift()).cumsum()).cumcount() + 1)
c(y.unstack()).unstack().T

In [ ]:
mu = df['retail'].mean()
sigma = df['retail'].std()
df_min = df['retail'].min()
df_max = df['retail'].max()
cut_div = [df_min, mu - 3*sigma, mu - 2*sigma, 
           mu - sigma, mu + sigma, mu + 2*sigma, mu + 3*sigma, df_max ]
df['retail_class'] = pd.cut(df['retail'], cut_div, labels=[1, 2, 3, 4, 5, 6, 7])
df[['retail', 'retail_class']].head(10)

In [ ]:
div_cat = np.arange(mu-3*sigma, mu+3*sigma, sigma).tolist() + [df_min, df_max]

In [ ]:
div_cat.sort()

In [ ]:
div_cat

In [ ]:
df.columns

In [ ]:
investors = ['date', 'retail', 'foreigner', 'institution', 'financial', 'invtrust', 'pension',
             'privequity', 'bank',  'insurance', 'financeetc', 'corporateetc', 'foreigneretc']

In [ ]:
df_inv = df[investors].copy()
df_inv_total = df_inv.sum(axis=1)
df_inv['total'] =  df_inv_total

In [ ]:
df_inv_total

In [ ]:
df_inv.head()

In [ ]:
# 7단계로 categorize using std. -3*std <, -2*std <, -1*std <, -1~1*std <, +1*std <, +2*std <, +3*std <
def segment_data(data):
    mu = data.mean()
    sigma = data.std()
    cut_div = [mu - 3*sigma, mu - 2*sigma, mu - sigma, mu + sigma, mu + 2*sigma, mu + 3*sigma ]
    msm3 =  mu - 3*sigma
    msm2 =  mu - 2*sigma
    if data 
    
titanic['age_class'] = pd.cut(titanic['age'], 3, labels=['child', 'young', 'old'])
titanic[['age', 'age_class']].head()

In [ ]:
retail = df['foreigner']
mu = retail.mean()
sigma = retail.std()

nbins = 10
n, bins, patches = plt.hist(retail, bins = nbins, density = 1)
plt.xlabel("foreigner")
plt.ylabel("Frequency")
plt.title("Distribution of retail")
y = ((1 / (np.sqrt(2 * np.pi) * sigma)) * np.exp(-0.5 * (1 / sigma * (bins - mu))**2))
# plt.plot(bins, y, '--' )
plt.plot(bins, y, '--', mu+3*sigma, 0.5e-5, '^', mu-3*sigma, 0.5e-5, '>' )
plt.plot( mu+2*sigma, 0.7e-5, '^', mu-2*sigma, 0.7e-5, '>' )
plt.plot( mu+1*sigma, 0.9e-5, '^', mu-1*sigma, 0.9e-5, '>' )
plt.show()

In [ ]:
retail = df['insurance']
mu = retail.mean()
sigma = retail.std()

nbins = 50
n, bins, patches = plt.hist(retail, bins = nbins, density = 1)
plt.xlabel("insurance")
plt.ylabel("Frequency")
plt.title("Distribution of retail")
y = ((1 / (np.sqrt(2 * np.pi) * sigma)) * np.exp(-0.5 * (1 / sigma * (bins - mu))**2))
# plt.plot(bins, y, '--' )
plt.plot(bins, y, '--', mu+3*sigma, 0.5e-5, '^', mu-3*sigma, 0.5e-5, '>' )
plt.plot( mu+2*sigma, 0.7e-5, '^', mu-2*sigma, 0.7e-5, '>' )
plt.plot( mu+1*sigma, 0.9e-5, '^', mu-1*sigma, 0.9e-5, '>' )
plt.show()

In [ ]:
plt.plot(mu+sigma, 1, '^')

In [ ]:
i=0
x = df['retail'][i:i+20]
i=2
y = df['retail'][i:i+20]
r = np.arange(20)
plt.plot(r, x)
plt.plot(r, y)

In [ ]:
np.arange(1, 21)

In [ ]:
for i in range(100-18):
    print(i, i+19)

In [ ]:
print(df.rolling(window=3).sum()) # 뒤에 추가 메서드를 이용하여 연산을 지정해주어야합니다.

In [ ]:
print(df.rolling(window=4, closed='left').sum())

In [ ]:
print(df.rolling(window=2, closed='right').sum())

In [ ]:
dict_test = {
    'col1': [1, 1, 2, 2, 3, 3, 3, 4],
    'col2': [1000, 1100, 2100, 2050, 3000, 3100, 3200, 4200],
    'col3': ['a', 'b', 'a', 'c', 'a', 'a', 'd', 'e']}

In [ ]:
df_test = pd.DataFrame(dict_test)

In [ ]:
df_test

In [ ]:
df_test[['col2']].rolling(3).sum()

In [ ]:
df_test[['col2']].rolling(3, center=True).sum()

In [ ]:
df_test.rolling(3, on= 'col2', center=True).sum()

In [ ]:
df_test[['col2']].rolling(3, min_periods=1, center=True).sum()

In [ ]:
df_test[['col2']].rolling(3, closed='left').sum()

In [ ]:
df_test[['col2']].rolling(3, closed='right').sum()

In [ ]:
df_test = df_test.groupby(by=['col1', 'col3'])['col2'].apply(list)

In [ ]:
df_test = pd.DataFrame(df_test)

In [ ]:
df_test

In [ ]:
df_test.reset_index(drop=False, inplace=True)

In [ ]:
df_test

In [ ]:
df_test.set_index(keys=['col1', 'col3'], inplace=True)

In [ ]:
df_test.reset_index(['col1'], inplace=True)